In [1]:
import json
import requests
from src import utils
import time
import os

In [2]:
api_path = '/userhomes/philhoon/kt-ai-challenge/api.json'

In [3]:
api = utils.read_json_file(api_path)

In [4]:
input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'
# in_path = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-train.jsonl'

In [5]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [6]:
os.makedirs(output_path, exist_ok=True)
output_file_name = 'result-' + input_file.split('/')[-1]
output_file = output_path + '/' + output_file_name
print(f'output_file : {output_file}')

output_file : /userhomes/philhoon/kt-ai-challenge/result/result-openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl


In [8]:
output_file = output_path + '/' + 'test.jsonl'
print(f'output_file : {output_file}')

output_file : /userhomes/philhoon/kt-ai-challenge/result/test.jsonl


In [9]:
jsonl_writer = utils.JSONLWriter(output_file)

In [10]:
data = utils.read_jsonlines(input_file)

In [11]:
# file_name

In [12]:
len(data)

1319

In [13]:
# api

In [14]:
# url = api['url']

In [15]:
# headers = api['headers']

In [16]:
# question = "질문 : 파운드 스털링은 가치가 있나요? 용어설명 :  사실목록 : ['1파운드는 법정 화폐입니다.', '법정 화폐는 정부 법령에 의해 뒷받침되며 내재적 가치가 없습니다', '2020년 5월 기준 1파운드 스털링은 약 1.24달러의 가치가 있습니다.']. 사실목록을 바탕으로 질문에 대한 답변을 사실 또는 거짓으로 대답하세요"/

In [17]:
# headers

In [18]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [19]:
model = model_api(api['url'], api['headers'])

In [20]:
BODY_TEMP = {
    "serviceInstanceId" : "ddm3kz9g", 
    "nluType" : "009",
    "apiType" : "002", 
    "utterance" : None
}

In [21]:
# body = json.dumps(body_temp)

In [22]:
# print(body)

In [23]:
# response = api.generate(body)

In [24]:
# context = response.json()['data'][0]['result'][0]

In [25]:
# print(context)

In [64]:
def get_gsm8k_zs(que):
    prompt = f"질문 : {que} 정답 : 질문를 단계별로 풀어보면 "
    return prompt

In [65]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [66]:
# from pprint import pprint

### GSM8K-TEST
    26 out of 1319 : contains the answer in the question due to translation
### GSM8K-TRAIN
    163 out of 7473 : contains the answer in the question due to translation

In [67]:
cnt = 0
split_cnt = 0
for ins in data:
    kor_que = ins['kor_question']
    
    # checking whether there is an answer 
    test = kor_que.split('\n')
    
    if len(test[0]) != len(kor_que):
        split_cnt += 1
        continue

    kor_ans = ins['kor_answer']
    prompt = get_gsm8k_zs(kor_que)
    ans_s = ins['answer_s']
    body = get_body(prompt, BODY_TEMP)
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]
    
    print(f'prompt : ')
    print(prompt)
    print(f'res_context : ')
    print(res_context)
    print(f'ans_s : ')
    print(ans_s)
    print('----')
    
    ins['kor_infer'] = res_context
#     jsonl_writer.write_json_line(ins)
    
#     print(prompt)
    if cnt == 20:
        break
    else:
        cnt += 1

/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문 : 한 벌의 로브를 만드는 데 파란색 섬유 2 덩이와 그 절반만큼의 흰색 섬유가 필요합니다. 총 몇 덩이의 섬유가 필요한가요? 정답 : 질문를 단계별로 풀어보면 
res_context : 
질문: 한 벌의 로브를 만드는 데 파란색 섬유 2 덩이와 그 절반만큼의 흰색 섬유가 필요합니다. 총 몇 덩이의 섬유가 필요한가요? 단계: 1. 섬유의 한 쪽을 파란색 섬유의 나머지 한 쪽으로 잡습니다. 2. 파란색 섬유의 나머지 한 쪽을 흰색 섬유의 나머지 한 쪽으로 잡습니다. 3. 파란색 섬유와 흰색 섬유를 각각 2덩이씩 잡습니다. 4. 총 섬유의 양을 계산합니다. 5. 2 + 3 = 5 6. 5 + 2 = 7 7. 7. 2 = 9 8. 9 = 12 9. 12 = 16 10. 16 = 24 11. 24 = 36 12. 36 = 48 13. 48 = 64 14. 48 = 64 15. 48 = 64 16. 48 = 64 17. 48 = 64 18. 48 = 64 19. 48 = 64 20. 48 = 64 21. 64 = 64 22. 64 = 64 23. 64 = 64 24. 64 = 64 25. 64 = 64 26. 64 = 64 27. 27 = 64 28. 28 = 64 29. 29 = 64 30. 30 = 64 31. 31 = 64 32. 32 = 64 33. 33 = 64 34. 34 = 64 35. 35 = 64 36. 36 = 64 37. 37 = 64 38. 38 = 64 39. 39 = 64 40. 40 = 64 41. 41 = 64 42. 42 = 64 43. 43 = 64 44. 44 = 64 45. 45 = 64 46. 46 = 64 47. 47 = 64 48. 48 = 64 49. 48
ans_s : 
3
----


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문 : 조쉬는 집을 뒤집어 판매해보기로 결정합니다. 그는 $80,000에 집을 구매하고 $50,000을 수리합니다. 이로 인해 집의 가치가 150% 증가합니다. 그가 얼마의 이익을 얻었습니까? 정답 : 질문를 단계별로 풀어보면 
res_context : 
조쉬는 집을 뒤집어 판매해 보기로 결정합니다. 조쉬는 $80,000에 집을 구매하고 $50,000을 수리합니다. 이로 인해 집의 가치가 150% 증가합니다. 조쉬는 집을 판매함으로써 얼마의 이익을 얻었습니까?
ans_s : 
70000
----


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

prompt : 
질문 : 마리사는 12마일 길을 하이킹하고 있습니다. 그녀는 처음 4마일을 걷는 데 1시간이 걸렸으며, 다음 2마일을 걷는 데 또 다른 1시간이 걸렸습니다. 그녀가 평균 속도를 시간당 4마일로 유지하려면, 남은 거리를 걷기 위해 어떤 속도(마일당 시간)로 걷어야 합니까? 정답 : 질문를 단계별로 풀어보면 
res_context : 
마리사는 12마일 길을 하이킹하고 있으며, 처음 4마일을 걷는 데 1시간이 걸렸습니다. 두 번째 2마일을 걷는 데 또 다른 1시간이 걸렸습니다. 평균 속도를 시간당 4마일로 유지하려면, 그녀는 남은 거리를 걷기 위해 다음과 같은 속도(마일당 시간)로 걸어야 합니다: 1시간 = 4마일/시간 = 4마일/시간/시간 = 2시간/시간 따라서, 그녀는 남은 거리를 걷기 위해 2시간/시간 (2마일/시간 x 2시간/시간)으로 걸어야 합니다.
ans_s : 
6
----


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문 : 저는 2/3이 물인 오렌지 음료를 10리터 가지고 있고, 3/5가 물인 파인애플 음료 15리터에 추가하려고 합니다. 그러나 따르는 도중에 오렌지 음료 1리터를 쏟았습니다. 남은 24리터에는 얼마나 많은 물이 있나요? 정답 : 질문를 단계별로 풀어보면 
res_context : 
1. 오렌지 음료 10리터를 2/3이 물인 파인애플 음료 15리터에 추가합니다. 2. 파인애플 음료 15리터에는 3/5가 물인 오렌지 음료가 15리터 포함되어 있습니다. 3. 따라서 파인애플 음료 15리터에는 2/3이 물인 오렌지 음료 10리터와 3/5가 물인 파인애플 음료 15리터가 포함되어 있습니다. 4. 따라서 남은 24리터에는 2/3이 물인 오렌지 음료 10리터와 3/5가 물인 파인애플 음료 15리터를 더하면 24리터 + (2/3 x 15리터) = 28리터가 됩니다. 5. 따라서 24리터에는 2/3이 물인 오렌지 음료 10리터와 3/5가 물인 파인애플 음료 15리터를 더하면 28리터가 됩니다.
ans_s : 
15
----


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문 : 레이몬드와 사만다는 사촌입니다. 레이몬드는 사만다보다 6년 먼저 태어났습니다. 레이몬드는 23살 때 아들을 가졌습니다. 사만다가 지금 31살이라면, 레이몬드의 아들이 태어난 지 몇 년이 지났을까요? 정답 : 질문를 단계별로 풀어보면 
res_context : 
1. 레이몬드는 사만다보다 6년 먼저 태어났습니다. 2. 레이몬드는 23살 때 아들을 가졌습니다. 3. 사만다가 지금 31살이라면, 레이몬드의 아들이 태어난 지 6년이 지났습니다. (사만다가 31세이므로, 레이몬드의 아들은 31세가 되기 전에 태어났습니다.)
ans_s : 
14
----
